In [4]:
import pandas as pd
from datetime import datetime
import os
from dbfread import DBF
def file_dirs(file_path):
    List = []
    for root, dirs, files in os.walk(file_path):
        if len(dirs) != 0:
            List += dirs
    return List


def file_files(file_path):
    List = []
    for root, dirs, files in os.walk(file_path):
        if len(files) != 0:
            List += files
    return List
zyml=pd.read_excel(r"C:\Py_Excel\Notebook\总表拼接\处理方法文件\2020年本科招生专业目录.xls")
zydic={}
for i in zyml.index:
    zydic[zyml['ZSZYMC'].at[i]]=zyml['YXBMMC'].at[i]
p1="C:/Py_Excel/Notebook/打印面单用/未发送"
List = file_dirs(p1)
result=pd.DataFrame([['考生号','姓名','学院','录取专业','家庭住址','通讯地址','联系电话','收件人','生源地','科类','批次']],index=[0],columns=['KSH','XM','XY','LQZY','JTDZ','TXDZ','LXDH','SJR','SYD','KL','PC'])
DT=pd.DataFrame([['考生号','姓名','学院','录取专业','家庭住址','通讯地址','联系电话','收件人','生源地','科类','批次']],index=[0],columns=['KSH','XM','XY','LQZY','JTDZ','TXDZ','LXDH','SJR','SYD','KL','PC'])
for x in List:
    path=p1
    path =os.path.join(path,x)
    strl = x.split()
    FilesList = file_files(path)
    tddxx = pd.read_excel(os.path.join(path,'t_tddxx.xls'),sheet_name='t_tddxx')
    if "T_BMK.dbf" in FilesList:
        # 导入BMK文件并作为df   表格对象
        table = DBF(os.path.join(path, "T_BMK.dbf"), encoding="GBK")
        df = pd.DataFrame(iter(table))
    else:
        table = DBF(os.path.join(path, "T_TDD.dbf"),
                    encoding="GBK",
                    char_decode_errors='ignore')
        df = pd.DataFrame(iter(table))
    re = pd.DataFrame(df[['KSH','XM','LXDH','SJR']])
    if 'JTDZ' in df.columns:
        re['JTDZ']=df['JTDZ']
    elif 'TXDZ' in df.columns:
        re['TXDZ']=df['TXDZ']
    re['KSH']=re['KSH'].astype('int64')
    
    dv = pd.merge(tddxx[['KSH','LQZY']],re,on='KSH')
    res = pd.concat([DT,dv],ignore_index=True)
    for i in res.iloc[1:,:].index:
        res["SYD"].at[i]=strl[0]
        res["KL"].at[i]=strl[2]
        res["PC"].at[i]=strl[1]
    result = pd.concat([result,res.iloc[1:,:]],ignore_index=True)

for i in result.iloc[1:,:].index:
    if '(三位一体)' in result['LQZY'].at[i]:
        result['LQZY'].at[i]=result['LQZY'].at[i][1:-6]
    else:
        result['LQZY'].at[i]=result['LQZY'].at[i][1:]
    result['XY'].at[i]=zydic[result['LQZY'].at[i]]

result['JTDZ'] = result['JTDZ'].fillna('')
result['TXDZ'] = result['TXDZ'].fillna('')
result['JTDZ'] = result['JTDZ']+result['TXDZ']
result=result.drop(columns=['TXDZ'])
result.to_excel(str(datetime.now().strftime("%m%d"))+'录取汇总.xlsx')


KeyError: 'YXBMMC'